In [1]:
import redcap
data = {
    'token': '55F9261859DDB94D0A7017A145361A75',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
proj = redcap.Project('https://redcap.mrc.gm:8443/redcaptest/api/',data['token'])

In [28]:
import pandas as pd
crf_name = 'ironsupplement'

# Use the export_records() function to export the specified CRF
crf_data = proj.export_records(forms=[crf_name])

crf_data=pd.DataFrame(crf_data)

In [29]:
counts = (
    (crf_data['iron_supferrouspri_q2'] == '1') |
    (crf_data['ip_administer'] == '1')
).groupby(crf_data['cons_recordid_q1']).sum().rename('Total_ips')


counts2 = (
    (crf_data['iron_supferrouspri_q2'] == '1') &
    (crf_data['ip_administer'] == '1')
).groupby(crf_data['cons_recordid_q1']).sum().rename('Total_ips_both')

total_counts = counts + counts2
total_counts.name = 'Total_ips_combined'

crf_data = pd.merge(crf_data, total_counts, on=['cons_recordid_q1'], how='left')
crf_data['IP']=84
crf_data['balance'] =crf_data['IP']- crf_data['Total_ips_combined']
crf_data.to_csv('IPlogTestData.csv',index=False)



In [30]:
import datetime

iron_supplement = proj.export_records(forms=['ironsupplement'])
iron_supplement = pd.DataFrame(iron_supplement)
iron_supplement = iron_supplement.rename(columns={'cons_recordid_q1': 'record_id'})

current_date = datetime.datetime.now().date()

max_supplement_date_str = iron_supplement['iron_patcontactdate_q1'].max()

if max_supplement_date_str:
    max_supplement_date = datetime.datetime.strptime(max_supplement_date_str, '%Y-%m-%d').date()
    days_difference = (current_date - max_supplement_date).days

    if days_difference > 1:
        iron_supplement.to_csv('missing.csv',index=False)
        print("The maximum iron supplement date is more than one day behind today.")


The maximum iron supplement date is more than one day behind today.


In [2]:
import re
import redcap
import pandas as pd
import pyodbc

# Connect to the SQL Server database
server = 'KB-DATA'
database = 'KenebaClinic'
driver = '{SQL Server}'
cnxn = pyodbc.connect(f"Driver={driver};Server={server};Database={database};Trusted_Connection=yes;")

# Retrieve data as Pandas DataFrame
sql_query = "SELECT * from vw_IdeA_sysmex"
sysmexresult_data = pd.read_sql(sql_query, cnxn)

# Format the values in the numeric columns to two decimal places
numeric_columns = ['sysm_wbc_q1', 'sysm_rbc_q2', 'sysm_hgb_q3']
sysmexresult_data[numeric_columns] = sysmexresult_data[numeric_columns].round(2)

# Define the desired float decimal format regex pattern
desired_pattern = r'^\d+\.\d{2}$'

# Validate the specified columns against the desired pattern
invalid_records = sysmexresult_data.copy()
invalid_records = invalid_records[~invalid_records[numeric_columns].astype(str).applymap(lambda x: bool(re.match(desired_pattern, x)))]

if not invalid_records.empty:
    print("Invalid records:")
    print(invalid_records)
else:
    # Remove the invalid records from sysmexresult_data
    sysmexresult_data = sysmexresult_data.drop(invalid_records.index)
    
    # Convert the filtered DataFrame to a list of dictionaries
    records_to_import = sysmexresult_data.to_dict('records')

    # Instantiate the REDCap project object
    token = '55F9261859DDB94D0A7017A145361A75'
    project_url = 'https://redcap.mrc.gm:8443/redcaptest/api/'
    proj = redcap.Project(project_url, token)

    # Import the valid records back to REDCap
    response = proj.import_records(records_to_import, return_content='ids', overwrite=True)

    # Check if the response contains any errors
    if 'error' in response:
        print("There was an error in importing the data.")
    else:
        print("Data has been successfully imported.")


Invalid records:
   cons_recordid_q1 sysm_wbc_q1 sysm_rbc_q2 sysm_hgb_q3 sysm_htc_q4  \
0               NaN         NaN         NaN         9.7         NaN   
1               NaN         NaN         NaN         9.9         NaN   
2               NaN         NaN         NaN        10.5         NaN   
3               NaN         NaN         NaN         NaN         NaN   
4               NaN         NaN         NaN        11.3         NaN   
..              ...         ...         ...         ...         ...   
78              NaN         NaN         NaN         7.7         NaN   
79              NaN         NaN         NaN         NaN         NaN   
80              NaN         NaN         NaN        10.5         NaN   
81              NaN         NaN         NaN         7.0         NaN   
82              NaN         NaN         NaN        10.4         NaN   

   sysm_mcv_q5 sysm_mch_q6 sysm_mchc_q7 sysm_plt_q8 sysm_rwd_sd_q9  ...  \
0          NaN         NaN          NaN         NaN    

C:\Users\abliebah\AppData\Local\Temp\ipykernel_19488\2773131569.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sysmexresult_data = pd.read_sql(sql_query, cnxn)


In [1]:
import redcap
import pandas as pd
import pyodbc
import redcap
data = {
    'token': '55F9261859DDB94D0A7017A145361A75',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
proj = redcap.Project('https://redcap.mrc.gm:8443/redcaptest/api/',data['token'])
server = 'KB-DATA'
database = 'KenebaClinic'
driver = '{SQL Server}'

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=KB-DATA;"
                      "Database=KenebaClinic;"
                      "uid=your_abliebah;"
                      "pwd=Mango@123;"
                      "Trusted_Connection=yes;")

# Retrieve data as Pandas DataFrame
sql_query = "SELECT * from vw_IdeA_sysmex"
sysmexresult_data = pd.read_sql(sql_query, cnxn)
sysmexresult_data = pd.DataFrame(sysmexresult_data)
records_to_import = sysmexresult_data.to_dict('records')
sysmexresult_data['sysmexresult_complete']=2
# Convert DataFrame records to a dictionary format
records_to_import = sysmexresult_data.to_dict('records')
response = proj.import_records(records_to_import, return_content='ids', overwrite=True)

    # Check if the response contains any errors
if 'error' in response:
        print("There was an error in importing the data.")
else:
        print("Data has been successfully imported.")


OperationalError: ('08001', '[08001] [Microsoft][SQL Server Native Client 11.0]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][SQL Server Native Client 11.0]Login timeout expired (0); [08001] [Microsoft][SQL Server Native Client 11.0]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

In [10]:
import json
import redcap
import pandas as pd
import pyodbc

# Connect to REDCap project
token = '55F9261859DDB94D0A7017A145361A75'
api_url = 'https://redcap.mrc.gm:8443/redcaptest/api/'
project = redcap.Project(api_url, token)

with open('config.json') as config_file:
    config = json.load(config_file)
username = config['username']
password = config['password']
server = config['server']
database = config['database']
driver = config['driver']
cnxn = pyodbc.connect(f"Driver={driver};Server={server};Database={database};uid={username};pwd={password};Trusted_Connection=yes;")
sql_query = "SELECT * FROM vw_IdeA_sysmex"
sysmexresult_data = pd.read_sql(sql_query, cnxn)
sysmexresult_data['sysmexresult_complete'] = 2
records_to_import = sysmexresult_data.to_dict('records')
event_name = 'v6_arm_1'  
for record in records_to_import:
    record['redcap_event_name'] = event_name

response = project.import_records(records_to_import, return_content='ids', overwrite=False)

# Check if the response contains any errors
if 'error' in response:
    print("There was an error in importing the data.")
else:
    print("Data has been successfully imported.")


C:\Users\abliebah\AppData\Local\Temp\ipykernel_19488\1952097006.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sysmexresult_data = pd.read_sql(sql_query, cnxn)


Data has been successfully imported.
